In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

# 1. Descripción del problema a resolver

En este cuadernillo se quiere cuantificar la brecha sanitaria que existe para diversos problemas
de salud relevantes para el Instituto Nacional del Tórax. En específico, se quiere saber si
tales problemas son resueltos por el sistema de salud o no. Para esto, se contrastará la información de 3 fuentes de información:

1. Casos teóricos por problema de salud según incidencia (ET).
2. Casos reales de egresos hospitalarios a nivel país por problema de salud (EP).
3. Casos reales de egresos hospitalarios del INT por problema de salud (EI).

En primer lugar, se calculará la brecha de egresos hospitalarios teóricos y a nivel país. O sea:

$Brecha Pais= ET - EP$

Si $ET > EP$, entonces existe una brecha, y el sistema sanitario ha fallado en resolver el problema de salud. En caso contrario, si se ha resuelto el problema de salud.

En nuestro caso, se quieren identificar los problemas de salud donde $Brecha Pais > 0$, ya que el INT podría hacerse cargo de tales casos sin resolver en el nuevo hospital.

Una vez identificado los problemas de salud donde hay capacidad de crecimiento, se calculará el porcentaje de crecimiento que tendría el INT al asumir el 100% de la brecha del país. Para esto, se utilizará la siguiente fórmula:

$Casos Nuevos del INT = EI + Brecha Pais$

$PorcentajeDeCrecimientoINT = \frac{Casos Nuevos del INT - EI}{EI} * 100$

Esto nos permitirá identificar qué diagnósticos necesitarán un mayor porcentaje de crecimiento que otros.

## Ejemplo de cálculo

A modo de ejemplo, se mostrará un cálculo manual:

$ET = 1055; EP = 493; EI = 220$

Con estos datos, podemos calcular que:

$Brecha Pais = 1055 - 493 = 562$

Por lo tanto, faltan **562** casos que el sistema sanitario falla en atender. Ahora, asumiendo que el INT se hará cargo del 100% de estos casos (sólo para temas de simplicidad. Sin embargo, en ciertos casos es imposible, especialmente tomando en cuenta la cantidad de casos que recibe actualmente el recinto), es posible calcular el % de crecimiento para esta brecha.

$Casos Nuevos del INT = 220 + 562 = 782$

$PorcentajeDeCrecimientoINT = \frac{782 - 220}{220} * 100 = 255\% $

Por lo tanto, el INT debería crecer en un 255% de su capacidad actual para suplir el 100% de la necesidad sanitaria en el diagnóstico X.


# 1.1 Cálculo de Brecha País con Egresos

## Extracción de egresos teóricos (ET)


In [3]:
# Carga la poblacion teorica de los 45 diagnosticos mas relevantes para el INT
poblacion_teorica = pd.read_excel(
    "../data/processed/proyeccion_problemas_de_salud.xlsx", sheet_name="Proyeccion problemas INE"
)

# Solamente deja el codigo CIE de los problemas de salud
poblacion_teorica["Diagnostico"] = poblacion_teorica.Diagnostico.str.split().str[0]

# Extrae los codigos de los diagnosticos relevantes y los deja como indice
DIAGS_RELEVANTES = poblacion_teorica.Diagnostico.unique()
poblacion_teorica = poblacion_teorica.set_index("Diagnostico").sort_index()

# De los casos totales, se asume que una parte (82%) es paciente FONASA. Ademas, se hara una
# correccion para estimar que son pacientes hospitalizados
RATIO_FONASA = 0.82
RATIO_HOSPITALIZADOS = 1
RATIO_AMBULATORIO = 1 - RATIO_HOSPITALIZADOS

In [4]:
# Deja los egresos teoricos a nivel pais
poblacion_teorica_pais = poblacion_teorica.query("Estrato == 'Pais'").copy()

# Selecciona los egresos teoricos hasta 2020
poblacion_teorica_pais = poblacion_teorica_pais[[i for i in range(2017, 2021)]]

# Corrige la poblacion teorica para pacientes FONASA y hospitalizados
poblacion_teorica_pais = round(poblacion_teorica_pais * RATIO_FONASA * RATIO_HOSPITALIZADOS)

## Extracción de egresos a nivel país (EP)


In [5]:
# Lee egresos hospitalarios a nivel país
egresos_pais = pd.read_csv(
    "../data/processed/ranking_nacional_egresos.csv",
    sep=";",
    encoding="latin-1",
    usecols=[
        "ANO_EGRESO",
        "ESTABLECIMIENTO_SALUD",
        "DIAG1",
        "n_egresos",
        "dias_estada_totales",
        "n_int_q",
        "n_muertos",
    ],
)

# Deja solamente los 45 egresos relevantes para el INT y desde 2017
egresos_pais = egresos_pais[egresos_pais["DIAG1"].isin(DIAGS_RELEVANTES)]
egresos_pais = egresos_pais.query("ANO_EGRESO >= 2017")

## Extracción de egresos INT (EI)


In [6]:
# Obtiene los egresos del Torax desde los egresos totales
egresos_int = egresos_pais.query("ESTABLECIMIENTO_SALUD == 112103").copy()

In [24]:
# Transforma la tabla de egresos del pais a formato de analisis
tabla_dinamica_egresos_pais = pd.pivot_table(
    egresos_pais,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos", "dias_estada_totales"],
    aggfunc="sum",
    fill_value=0,
).sort_index()


# Transforma la tabla de egresos int a formato de analisis

tabla_dinamica_egresos_int = pd.pivot_table(
    egresos_int,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos", "dias_estada_totales"],
    aggfunc="sum",
    fill_value=0,
).sort_index()

# tabla_dinamica_egresos_pais.columns = tabla_dinamica_egresos_pais.columns.droplevel(0)
# tabla_dinamica_egresos_int.columns = tabla_dinamica_egresos_int.columns.droplevel(0)

Una vez extraído los datos y haberlos transformados para realizar su análisis, se procederá a calcular las respectivas brechas.

En primer lugar, se obtendrá la brecha a nivel país por problema de salud. Recordemos que la fórmula de cálculo es:

$Brecha Pais= ET - EP$


In [31]:
brecha_pais = (poblacion_teorica_pais - tabla_dinamica_egresos_pais[("n_egresos")]).astype("Int32")
brecha_pais.style.map(visualize.color_negative_red)

,2017,2018,2019,2020
Diagnostico,,,,
C33X,14,22,24,27
C340,879,918,933,998
C341,670,583,588,785
C342,992,993,1029,1053
C343,784,768,802,878
C381,140,144,136,149
C384,-34,-39,-47,-37
C450,-76,-71,-70,-47
C780,669,682,651,775


Los resultados indican que en gran parte de los diagnósticos faltó suplir la demanada del problema
de salud (ej: C340, C341, C342, C343, etc). Los resultados se pueden dividir en distintos tipos
de
problema de salud:

1. **Cáncer de Pulmón (C33X a C782)**: En todos los diagnósticos faltó suplir la necesidad del
   país (exceptuando C33X, C384, C450). EL diagnóstico con una mayor necesidad de atención fue
   el C342, con una falta de 1026 egresos.

2. **Patologías Valvulares (I051 a I712)**: En todos los diagnósticos faltó suplir la necesidad del
   país (exceptuando I420).


Con estos resultados, es posible identificar todos los diagnósticos donde es necesario cerrar la brecha de atención de salud. Se seleccionaran los diagnósticos donde en al menos 1 de los años entre 2017 y 2020 exista una $Brecha País > 0.$


In [32]:
# Selecciona solamente los diagnosticos donde exista una brecha > 0 en alguno de los anios
brecha_pais_a_resolver = brecha_pais[(brecha_pais > 0).any(axis=1)]
# Obtiene los codigos de los diagnosticos con una brecha
diagnosticos_con_necesidad_sanitaria = brecha_pais_a_resolver.index

In [35]:
print(f"Los diagnósticos que tienen una necesidad sanitaria a nivel país son:\n"
      f"{list(diagnosticos_con_necesidad_sanitaria)}")

Los diagnósticos que tienen una necesidad sanitaria a nivel país son:
['C33X', 'C340', 'C341', 'C342', 'C343', 'C381', 'C780', 'C782', 'D143', 'D381', 'I051', 'I340', 'I350', 'I456', 'I472', 'I495', 'I710', 'I712', 'J47X', 'J679', 'J841', 'J848', 'J849', 'J90X', 'J91X', 'J931', 'M348', 'Q676']


# 1.2 Cálculo de Hospital Basal tomando todos los casos del SSMO

En este apartado, luego de haber identificado los diagnósticos donde existe una brecha de atención,
se estimará la cantidad de casos que deberá atender el INT si es que recibe el 100% de la brecha de pacientes en el SSMO. Dicho en otras palabras, se calculará la necesidad de recursos si es que el INT atiende al 100% de su población en el SSMO.

Para esto, se obtendrá la **población teórica de los diagnósticos donde haya brecha de atención en el país**. Luego, se sumarán estos casos a los egresos INT del 2019. Finalmente, con la cantidad de egresos nuevos, se estimarán los dias de estada necesarios, y la cantidad de camas necesarias para suplir tales días. O sea:

$Población Teórica SSMO (PTSSMO) + Egresos 2019 INT = Casos Nuevos INT$

$Cantidad de Camas Necesarias = \frac{(Casos Nuevos INT * Dias de Estada Promedio Diagnostico)}{365}$

In [13]:
# Los diagnosticos con necesidad sanitaria estan en 
# la variable `diagnoticos_con_necesidad_sanitaria`

# Deja solamente la poblacion teorica del SSMO
poblacion_teorica_ssmo = poblacion_teorica.query("Estrato == 'SSMO'")

# Selecciona los egresos teoricos hasta 2035
poblacion_teorica_ssmo = poblacion_teorica_ssmo[[i for i in range(2017, 2021)]]

# Corrige por la poblacion de FONASA y hospitalizados
poblacion_teorica_ssmo = round(poblacion_teorica_ssmo * RATIO_FONASA * RATIO_HOSPITALIZADOS)

In [14]:
# Aqui se deja un espacio para calcular la brecha a nivel de servicio. Es necesario obtener los
# egresos agrupados a nivel de servicio, para determinar si existe una brecha de atencion a nivel
# de servicio.

# Si se utiliza la brecha nacional, y se obtiene un porcentaje de esta para estimar la brecha del
# servicio, seria una estimacion poco exacta.
HOSPITALES_SSMO = [
    112300,
    112606,
    112609,
    112612,
    112607,
    112608,
    112610,
    112611,
    112613,
    112102,
    112100,
    112101,
    112107,
    114104,
    112104,
    112103,
    112105,
    112106,
]

In [37]:
# Aisla solamente los recintos del SSMO, y calcula los egresos y dias de estada para los diags 
# relevantes

egresos_ssmo = egresos_pais.query("ESTABLECIMIENTO_SALUD.isin(@HOSPITALES_SSMO)")
tabla_dinamica_egresos_ssmo = pd.pivot_table(
    egresos_ssmo,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos", "dias_estada_totales"],
    aggfunc="sum",
    fill_value=0,
).sort_index()

In [48]:
# Calcula la brecha de casos para el SSMO
brecha_ssmo = (poblacion_teorica_ssmo - tabla_dinamica_egresos_ssmo[("n_egresos")]).astype("Int32")
display(brecha_ssmo.style.map(visualize.color_negative_red))

# Ahora, todos los casos donde se ha cumplido la necesidad sanitaria, se reemplazaran por 0
brecha_ssmo[(brecha_ssmo < 0) | (brecha_ssmo.isna())] = 0

,2017,2018,2019,2020
Diagnostico,,,,
C33X,2,2,-1,-5
C340,54,59,57,76
C341,-107,-145,-140,5
C342,56,54,60,76
C343,-44,-46,-42,44
C381,-2,-7,-5,9
C384,-9,-10,-11,-4
C450,-23,-27,-8,-5
C780,-6,5,3,57


In [54]:
# Calcula los nuevos casos del INT si es que asume el 100% de la brecha del SSMO.
nuevos_casos_int_con_brecha_ssmo = tabla_dinamica_egresos_int["n_egresos"] + brecha_ssmo

In [59]:
# Extrae la cantidad de dias estada promedio del INT
dias_estada_promedio_int = (
    tabla_dinamica_egresos_int["dias_estada_totales"] / tabla_dinamica_egresos_int["n_egresos"]
)

In [64]:
# Calcula la cantidad de camas que necesitó el INT
cantidad_camas_antiguas_int = (tabla_dinamica_egresos_int["dias_estada_totales"] / 365)

In [73]:
dias_estada_estimados_int_con_brecha_ssmo = (
    nuevos_casos_int_con_brecha_ssmo * dias_estada_promedio_int
)
cantidad_camas_nuevas_int = (dias_estada_estimados_int_con_brecha_ssmo) / 365

In [75]:
cantidad_camas_antiguas_int.sum()

ANO_EGRESO
2017    59.454795
2018    61.164384
2019    59.684932
2020    33.624658
dtype: float64

In [76]:
cantidad_camas_nuevas_int.sum()

ANO_EGRESO
2017    302.654995
2018    434.518428
2019    328.011496
2020    319.732954
dtype: Float64